In [1]:
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub
import tensorflow_text as text
import pickle

2024-08-06 00:27:36.275515: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-06 00:27:36.510238: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-06 00:27:36.556789: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-06 00:27:36.557159: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-06 00:27:36.668355: I tensorflow/core/platform/cpu_feature_gua

In [2]:
#import data
headlines_annotated = pd.read_csv("./data/headlines_annotated.tsv", sep="\t")

In [3]:
#select only relevant columns
headlines_annotated = headlines_annotated[['headline', 'dominant_emotion']]

In [4]:
headlines_annotated

,headline,dominant_emotion
0,Cops in One Village Have Been Convicted of 70 ...,anger
1,DIY penis enlargements are a 'nationwide probl...,negative_surprise
2,Dam breaking: New Epstein accuser comes forward,anger
3,David Beckham gets six-month driving ban for u...,negative_surprise
4,Dead sea turtle found with spear through head ...,sadness
...,...,...
4995,‘NOW do abortion. We’ll wait.’ Ilhan Omar doub...,negative_surprise
4996,‘Triad’ Thugs Use Clubs to Punish Hong Kong’s ...,negative_surprise
4997,'For real': High school football season begins...,positive_anticipation_including_optimism
4998,100 years on: the picture that changed our vie...,positive_surprise


In [5]:
headlines_annotated["dominant_emotion"].value_counts()

dominant_emotion
negative_surprise                            893
positive_surprise                            582
anger                                        460
annoyance                                    426
fear                                         419
sadness                                      386
disgust                                      382
negative_anticipation_including_pessimism    323
positive_anticipation_including_optimism     319
joy                                          264
guilt                                        175
trust                                        124
shame                                        119
pride                                         65
love_including_like                           63
Name: count, dtype: int64

In [6]:
#replace emotions with optimism and pessimism

replace_with_optimisim = ["positive_surprise", "positive_anticipation_including_optimism"]

replace_with_pessimism = ["negative_surprise", "negative_anticipation_including_pessimism"]


optimism_dict = {k: "optimism" for k in replace_with_optimisim}

pessimism_dict = {k: "pessimism" for k in replace_with_pessimism}

headlines_annotated["dominant_emotion"] = headlines_annotated["dominant_emotion"].replace(optimism_dict)
headlines_annotated["dominant_emotion"] = headlines_annotated["dominant_emotion"].replace(pessimism_dict)

headlines_annotated["dominant_emotion"].value_counts()

dominant_emotion
pessimism              1216
optimism                901
anger                   460
annoyance               426
fear                    419
sadness                 386
disgust                 382
joy                     264
guilt                   175
trust                   124
shame                   119
pride                    65
love_including_like      63
Name: count, dtype: int64

In [7]:
#see value counts
headlines_annotated["dominant_emotion"].value_counts()

dominant_emotion
pessimism              1216
optimism                901
anger                   460
annoyance               426
fear                    419
sadness                 386
disgust                 382
joy                     264
guilt                   175
trust                   124
shame                   119
pride                    65
love_including_like      63
Name: count, dtype: int64

In [8]:
#undersample the majority class
rus = RandomUnderSampler(random_state=69)
X_resampled, y_resampled = rus.fit_resample(headlines_annotated["headline"].values.reshape(-1, 1), headlines_annotated["dominant_emotion"].values)


In [9]:
#merge the two arrays back into pandas dataframe
headlines_resampled = pd.DataFrame(np.hstack((X_resampled, y_resampled.reshape(-1, 1))), columns=["headline", "dominant_emotion"])


In [10]:
#checking the length of X
len(X_resampled)

819

In [11]:
#split into training, testing, and validation sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=69)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=69)

In [12]:
#merge the X and ys together
train = np.concatenate((X_train, y_train.reshape(-1, 1)), axis=1)
test = np.concatenate((X_test, y_test.reshape(-1, 1)), axis=1)
val = np.concatenate((X_val, y_val.reshape(-1, 1)), axis=1) 

In [13]:
#export to csv
np.savetxt("./data/train.csv", train, delimiter=",", fmt="%s")
np.savetxt("./data/test.csv", test, delimiter=",", fmt="%s")
np.savetxt("./data/val.csv", val, delimiter=",", fmt="%s")
